In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !python3 -m pip install einops
# !python3 -m pip install facenet-pytorch
# !python3 -m pip install face_alignment
# !python3 -m pip install self_attention_cv

In [3]:
import cv2
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
from einops.layers.torch import Rearrange
from torchvision import datasets
from torch.utils.data import DataLoader
from transformers import ViTModel, ViTConfig
from einops import rearrange
from facenet_pytorch import MTCNN
from transformers.modeling_outputs import ImageClassifierOutput  # Corrected import
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
import face_alignment
from transformers.models.vit.modeling_vit import ViTForImageClassification
#import dlib
import requests
from torchvision.transforms import functional as F

/home1/riyaranj/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def extract_frame(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    mid_frame_index = frame_count // 2  # Index of the frame in the middle of the video
    cap.set(cv2.CAP_PROP_POS_FRAMES, mid_frame_index)
    ret, frame = cap.read()
    if ret:
        cap.release()
        return frame
    else:
        cap.release()
        return None

In [5]:
def detect_face(frame):
    mtcnn = MTCNN()
    boxes, _ = mtcnn.detect(frame)
    if boxes is not None:
        # Assuming only one face in the frame
        box = boxes[0]
        x1, y1, x2, y2 = box
        # Crop the frame to the detected face
        cropped_frame = frame[int(y1):int(y2), int(x1):int(x2)]
        return cropped_frame
    else:
        return None

In [6]:
# Function to download the pretrained face alignment model if it doesn't exist
def download_face_alignment_model(url, save_path):
    if not os.path.exists(save_path):
        print("Downloading pretrained face alignment model...")
        response = requests.get(url)
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print("Download complete.")

# Specify the URL of the pretrained face alignment model
face_alignment_model_url = "https://github.com/1adrianb/face-alignment-models/releases/download/2.0.1/2DFAN4-11f355bf06.pth.tar"

# Download the pretrained face alignment model if it doesn't exist
face_alignment_model_path = os.path.abspath("2DFAN4-11f355bf06.pth.tar")
download_face_alignment_model(face_alignment_model_url, face_alignment_model_path)

# Initialize face alignment model
fa = face_alignment.FaceAlignment(2, flip_input=False)  # 2 corresponds to 2D landmarks

def align_face(frame):
    # Perform face alignment
    aligned_faces = fa.get_landmarks(frame)
    if aligned_faces is not None:
        aligned_face = aligned_faces[0]  # Assuming only one face in the frame
        return aligned_face
    else:
        return None


In [7]:
def preprocess_image(frame):
    frame_pil = Image.fromarray(frame.astype('uint8'))
    frame_pil = frame_pil.convert('RGB')  # Convert to RGB format
    frame_np = np.array(frame_pil)
    print(frame_np.shape)
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize using ImageNet mean and std
    ])
    frame_tensor = transform(frame_pil)
    return frame_tensor


In [8]:
def preprocess_spectrogram(image_path):
    img = Image.open(image_path).convert('RGB')  # Convert to grayscale
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to match ViT input size
        transforms.ToTensor(),           # Convert to tensor
    ])
    img_tensor = transform(img)
    return img_tensor

In [9]:
def load_spectrogram_dataset(spectrograms_folder, skipped_files):
    X = []
    y = []
    # List all files in the input folder
    files = sorted(os.listdir(spectrograms_folder))
    # Iterate over files in the folder
    for filename in tqdm(files):
        if filename.endswith(".png") and filename[:-3] not in skipped_files:  # Assuming mel spectrograms are stored as PNG files
            input_path = os.path.join(spectrograms_folder, filename)
            img_tensor = preprocess_spectrogram(input_path)
            X.append(img_tensor)
            # Extract label from filename (assuming filename is in format "abc_IEO_label_xyz.png")
            label = filename.split("_")[2]
            if label == "HAP":
                y.append(0)
            elif label == "SAD":
                y.append(1)
            elif label == "ANG":
                y.append(2)
            elif label == "DIS":
                y.append(3)
            elif label == "FEA":
                y.append(4)
            elif label == "NEU":
                y.append(5)
    return X, y

In [10]:
def load_dataset(videos_folder):
    X = []
    y = []
    skipped_files = []
    video_files = [file for file in sorted(os.listdir(videos_folder)) if file.endswith(".flv")]
    for video_file in tqdm(video_files):
        video_path = os.path.join(videos_folder, video_file)
        frame = extract_frame(video_path)
        if frame is not None:
            cropped_face = detect_face(frame)
            if cropped_face is not None:
                preprocessed_face = preprocess_image(cropped_face)
                X.append(preprocessed_face)
                label = video_file.split("_")[2].split(".")[0]  # Adjusted to handle different file extensions
                if label == "HAP":
                    y.append(0)
                elif label == "SAD":
                    y.append(1)
                elif label == "ANG":
                    y.append(2)
                elif label == "DIS":
                    y.append(3)
                elif label == "FEA":
                    y.append(4)
                elif label == "NEU":
                    y.append(5)
            else:
                print(f"No face detected in {video_file}. Skipping.")
                skipped_files.append(video_file[:-3])
        else:
            print(f"Failed to extract frame from {video_file}. Skipping.")
            skipped_files.append(video_file[:-3])
    return X, y, skipped_files

In [11]:
# Define the ConcatDataset class to concatenate video frame and spectrogram tensors
class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, X1, X2, y, modality='multimodal', fullscale=False, presaved=False, target_size=(224,224)):
        self.X1 = X1
        self.X2 = X2
        self.y = y
        self.modality = modality
        self.presaved = presaved
        self.fullscale = fullscale
        self.target_size = target_size
        
    def __len__(self):
        if self.modality == 'audio':
          return len(self.X2)
        if self.modality == 'visual':
          return len(self.X1)
        return len(self.y)

    def __getitem__(self, idx):
        if not self.fullscale:
          img1 = self.X1[idx]
          img2 = self.X2[idx]
          label = self.y[idx]
        else:
          img1 = torch.from_numpy(self.X1[idx]).float()  # Convert numpy array to torch tensor
          img2 = torch.from_numpy(self.X2[idx]).float()  # Convert numpy array to torch tensor
          label = torch.tensor(self.y[idx])  # Convert numpy array to torch tensor

        concatenated_img = torch.cat((img1, img2), dim=1)  # Concatenate along 1 dimension
        concatenated_img = F.resize(concatenated_img, self.target_size)
        #print(concatenated_img.shape)
        
        if self.modality == 'visual':
          return img1, label
        if self.modality == 'audio':
          return img2, label
        return concatenated_img, label # concatenate modalities

In [12]:
def train_model(model, criterion, optimizer, train_loader, device):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)

        # Extract logits if the model output is wrapped in an ImageClassifierOutput object
        if isinstance(outputs, ImageClassifierOutput):
            logits = outputs.logits
        else:
            # Handle other cases
            pass
        
        # Print the size of the logits tensor
        #print("Logits size:", logits.size())
        
        labels = labels.long()  # Convert labels to long type for CrossEntropyLoss
        loss = criterion(logits, labels)  # Calculate loss using entire logits
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predicted = torch.argmax(logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return total_loss / len(train_loader), correct / total


In [13]:
def test_model(model, criterion, test_loader, device):
    model.eval()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            labels = labels.long()
            loss = criterion(outputs.logits, labels)  # Calculate loss using entire logits
            running_loss += loss.item() * inputs.size(0)
            predicted = torch.argmax(outputs.logits, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)
    
    epoch_loss = running_loss / len(test_loader.dataset)
    accuracy = correct_preds / total_preds
    return epoch_loss, accuracy

In [14]:
# class ViT(nn.Module):
#     # ViT architecture adapted from here - https://theaisummer.com/vision-transformer/
#     def __init__(self, *,
#                  img_dim,
#                  in_channels=3,
#                  patch_dim=16,
#                  num_classes=6, # full-scale CREMA-D
#                  dim=512,
#                  blocks=6,
#                  heads=4,
#                  dim_linear_block=1024,
#                  dim_head=None,
#                  dropout=0, transformer=None, classification=True):
#         """
#         Args:
#             img_dim: the spatial image size
#             in_channels: number of img channels
#             patch_dim: desired patch dim
#             num_classes: classification task classes
#             dim: the linear layer's dim to project the patches for MHSA
#             blocks: number of transformer blocks
#             heads: number of heads
#             dim_linear_block: inner dim of the transformer linear block
#             dim_head: dim head in case you want to define it. defaults to dim/heads
#             dropout: for pos emb and transformer
#             transformer: in case you want to provide another transformer implementation
#             classification: creates an extra CLS token
#         """
#         super().__init__()
#         assert img_dim % patch_dim == 0, f'patch size {patch_dim} not divisible'
#         self.p = patch_dim
#         self.classification = classification
#         tokens = (img_dim // patch_dim) ** 2
#         self.token_dim = in_channels * (patch_dim ** 2)
#         self.dim = dim
#         self.dim_head = (int(dim / heads)) if dim_head is None else dim_head
#         self.project_patches = nn.Linear(self.token_dim, dim)

#         self.emb_dropout = nn.Dropout(dropout)
#         if self.classification:
#             self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
#             self.pos_emb1D = nn.Parameter(torch.randn(tokens + 1, dim))
#             self.mlp_head = nn.Linear(dim, num_classes)
#         else:
#             self.pos_emb1D = nn.Parameter(torch.randn(tokens, dim))

#         if transformer is None:
#             self.transformer = TransformerEncoder(dim, blocks=blocks, heads=heads,
#                                                   dim_head=self.dim_head,
#                                                   dim_linear_block=dim_linear_block,
#                                                   dropout=dropout)
#         else:
#             self.transformer = transformer

#     def expand_cls_to_batch(self, batch):
#         """
#         Args:
#             batch: batch size
#         Returns: cls token expanded to the batch size
#         """
#         return self.cls_token.expand([batch, -1, -1])

#     def forward(self, img, mask=None):
#         batch_size = img.shape[0]
#         img_patches = rearrange(
#         img, 'b c (patch_x x) (patch_y y) -> b (x y) (patch_x patch_y c)',
#         patch_x=self.p, patch_y=self.p
#     )
#     # project patches with linear layer + add pos emb
#         img_patches = self.project_patches(img_patches)

#         if self.classification:
#             img_patches = torch.cat(
#             (self.expand_cls_to_batch(batch_size), img_patches), dim=1
#         )

#         patch_embeddings = self.emb_dropout(img_patches + self.pos_emb1D)

#     # feed patch_embeddings and output of transformer. shape: [batch, tokens, dim]
#         y = self.transformer(patch_embeddings, mask)

#         if self.classification:
#         # we index only the cls token for classification.
#             return self.mlp_head(y[:, 0, :])
#         else:
#             return y  # Return logits directly
#     # def forward(self, img, mask=None):
#     #     batch_size = img.shape[0]
#     #     img_patches = rearrange(
#     #         img, 'b c (patch_x x) (patch_y y) -> b (x y) (patch_x patch_y c)',
#     #                             patch_x=self.p, patch_y=self.p)
#     #     # project patches with linear layer + add pos emb
#     #     img_patches = self.project_patches(img_patches)

#     #     if self.classification:
#     #         img_patches = torch.cat(
#     #             (self.expand_cls_to_batch(batch_size), img_patches), dim=1)

#     #     patch_embeddings = self.emb_dropout(img_patches + self.pos_emb1D)

#     #     # feed patch_embeddings and output of transformer. shape: [batch, tokens, dim]
#     #     y = self.transformer(patch_embeddings, mask)

#     #     if self.classification:
#     #         # we index only the cls token for classification.
#     #         return self.mlp_head(y[:, 0, :])
#     #     else:
#     #         return y

In [15]:
_fullscale = True # Run fullscale experiment?
_presaved = True # Use presaved data .npy files?

# Define input_folder and input_folder_spec
if _fullscale:
  input_folder = '/home1/riyaranj/riya/videos_fullscale'
  input_folder_spec = '/home1/riyaranj/riya/melspec_fullscale'
else:
  input_folder = '/home1/riyaranj/riya/videos'
  input_folder_spec = '/home1/riyaranj/riya/melspec'

# Check if input folder exists
if not os.path.exists(input_folder):
    print("Input folder does not exist.")
    sys.exit(1)
# Check if input folder exists
if not os.path.exists(input_folder_spec):
    print("Input folder does not exist.")
    sys.exit(1)

#### Check that the videos and spectrograms are in one-to-one correspondence

In [16]:
assert([file[:-3] for file in sorted(os.listdir(input_folder))] == [file[:-3] for file in sorted(os.listdir(input_folder_spec))])

In [17]:
if _presaved:
  X = np.load('/home1/riyaranj/riya/X_7439.npy', mmap_mode='r')
  y = np.load('/home1/riyaranj/riya/y_7439.npy', mmap_mode='r')
  X_spec = np.load('/home1/riyaranj/riya/X_spec_7439.npy', mmap_mode='r')
  y_spec = np.load('/home1/riyaranj/riya/y_spec_7439.npy', mmap_mode='r')
else:
    X, y, skipped_files = load_dataset(input_folder)
    X_spec, y_spec = load_spectrogram_dataset(input_folder_spec, skipped_files)

# Split the data into train and test sets
print(f"Total number of samples: {len(X)}")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(f"Number of train samples (video): {len(X_train)}", f"Number of test samples: {len(X_test)}")
X_train_spec, X_test_spec, y_train_spec, y_test_spec = train_test_split(X_spec, y_spec, test_size=0.3, random_state=42)
print(f"Number of train samples (audio): {len(X_train_spec)}", f"Number of test samples: {len(X_test_spec)}")

Total number of samples: 7429
Number of train samples (video): 5200 Number of test samples: 2229
Number of train samples (audio): 5200 Number of test samples: 2229


#### Save data as .npy files (if needed)

In [18]:
# # Save X, y, X_spec, y_spec
# np.save('X_7439.npy', np.array(X))
# np.save('y_7439.npy', np.array(y))
# np.save('X_spec_7439.npy', np.array(X_spec))
# np.save('y_spec_7439.npy', np.array(y_spec))

#### Check that the number of frames and spectrograms are equal and that the train + test samples add up to total samples

In [19]:
assert(len(X_train) == len(X_train_spec) and len(X_test) == len(X_test_spec))

In [20]:
def train_ViT(_modality):
    best_test_accuracy = 0.0
    best_results = None
    best_model_state = None

    batch_sizes = [32]
    learning_rates = [0.0001]

    for _bs in batch_sizes:
        for _lr in learning_rates:
            # Adjust input channels as per modality
            if _modality == 'multimodal':
                _input_channels = 2
            else:
                _input_channels = 1

            # Initialize the ViT model
            config = ViTConfig.from_pretrained('google/vit-base-patch16-224-in21k')
            config.num_labels = 6
            model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k', config=config, ignore_mismatched_sizes=True)

            # Define device
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)

            # Define loss function and optimizer
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=_lr)

            # Concatenate datasets if multimodal
            train_dataset = ConcatDataset(X_train, X_train_spec, y_train, modality=_modality, fullscale = _fullscale)
            test_dataset = ConcatDataset(X_test, X_test_spec, y_test, modality=_modality, fullscale = _fullscale)

            # Create data loaders
            train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=_bs, shuffle=True)
            test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=_bs)

            print(f"\n\nBatch size: {_bs}, lr: {_lr}")

            # Training loop
            num_epochs = 30
            print(f"Training ViT for \"{_modality}\" pipeline ...\n------------------------------------------------\n")
            for epoch in range(num_epochs):
                print("Epoch " + str(epoch))
                train_loss, train_accuracy = train_model(model, criterion, optimizer, train_loader, device)
                test_loss, test_accuracy = test_model(model, criterion, test_loader, device)
                print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

                # Update best model and results if test accuracy improves
                if test_accuracy > best_test_accuracy:
                    best_test_accuracy = test_accuracy
                    best_results = (train_loss, train_accuracy, test_loss, test_accuracy, _bs, _lr)
                    best_model_state = model.state_dict()

            # Save the model
            if _modality == 'multimodal':
                torch.save(model.state_dict(), f'HF_ViT_audio_video_fullscale_DIM1_{num_epochs}_{_bs}_{_lr}.pt')
            elif _modality == 'audio':
                torch.save(model.state_dict(), f'HF_ViT_audio_fullscale_DIM1_{num_epochs}_{_bs}_{_lr}.pt')
            elif _modality == 'visual':
                torch.save(model.state_dict(), f'HF_ViT_video_fullscale_DIM1_{num_epochs}_{_bs}_{_lr}.pt')
            else:
                print("Improper modality provided!")

    print("Best Results:")
    print(f"Train Loss: {best_results[0]}, Train Accuracy: {best_results[1]}, Test Loss: {best_results[2]}, Test Accuracy: {best_results[3]}, Batch Size: {best_results[4]}, Learning Rate: {best_results[5]}")

    return best_results



#### Choose modalities

In [21]:
# # Define modalities
_modality = ['visual', 'audio', 'multimodal']

In [22]:
#### Train ViT

In [23]:

# Train ViT
for _m in _modality:
    train_ViT(_m)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.




Batch size: 32, lr: 0.0001
Training ViT for "visual" pipeline ...
------------------------------------------------

Epoch 0


100%|██████████| 70/70 [00:14<00:00,  4.88it/s]


Epoch 1/30, Train Loss: 1.3392, Train Accuracy: 0.4663, Test Loss: 1.0887, Test Accuracy: 0.5617
Epoch 1


100%|██████████| 70/70 [00:14<00:00,  4.90it/s]


...

Epoch 28/30, Train Loss: 0.0254, Train Accuracy: 0.9913, Test Loss: 0.8325, Test Accuracy: 0.7963
Epoch 28


100%|██████████| 70/70 [00:14<00:00,  4.92it/s]


Epoch 29/30, Train Loss: 0.0193, Train Accuracy: 0.9933, Test Loss: 0.9226, Test Accuracy: 0.8008
Epoch 29


100%|██████████| 70/70 [00:14<00:00,  4.92it/s]


Epoch 30/30, Train Loss: 0.0337, Train Accuracy: 0.9890, Test Loss: 1.0044, Test Accuracy: 0.7707
Best Results:
Train Loss: 0.03979573138715253, Train Accuracy: 0.9867307692307692, Test Loss: 0.7543248275621636, Test Accuracy: 0.82907133243607, Batch Size: 32, Learning Rate: 0.0001
